In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch.nn.utils.parametrizations import weight_norm
import time
import pandas as pd
import random
from sklearn.metrics import r2_score

# Set random seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)  # Set the random seed to 42

# Set the font for matplotlib to support Chinese characters
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# Function to transform predicted and true values
def transform_values(psi):
    return (1 / 0.008) * torch.log(psi + torch.exp(torch.tensor(-4.0)))

# Forward gradient function
def fwd_gradients(Y, x):
    dummy = torch.ones_like(Y)
    G = torch.autograd.grad(Y, x, dummy, create_graph=True)[0]
    return G

# Neural network class with Xavier initialization and weight normalization
class Net(torch.nn.Module):
    def __init__(self, layer_dim, X, device):
        super().__init__()
        self.X_mean = torch.from_numpy(X.mean(0, keepdims=True)).float().to(device)
        self.X_std = torch.from_numpy(X.std(0, keepdims=True)).float().to(device)

        self.num_layers = len(layer_dim)
        temp = []
        for l in range(1, self.num_layers):
            linear = weight_norm(torch.nn.Linear(layer_dim[l-1], layer_dim[l]), dim=0)
            torch.nn.init.xavier_uniform_(linear.weight)
            if linear.bias is not None:
                torch.nn.init.zeros_(linear.bias)
            temp.append(linear)
        self.layers = torch.nn.ModuleList(temp)

    def forward(self, x):
        x = (x - self.X_mean) / self.X_std  # Normalize input
        for i in range(0, self.num_layers-1):
            x = self.layers[i](x)
            if i < self.num_layers-2:
                x = F.sigmoid(x)
        return x

# TSONN class for training and model management
class TSONN:
    def __init__(self, layers, device):
        self.layers = layers
        self.device = device

        Nx = 101  # Number of grid points
        x = torch.linspace(0, 10, Nx).to(self.device)
        self.X_ref = x.reshape(-1, 1)

        # Boundary conditions at x = 0 and x = 10
        self.X_lbc = torch.tensor([[0]], device=self.device)
        self.X_ubc = torch.tensor([[10.0]], device=self.device)

        self.Nx = Nx
        self.log = {'losses': [], 'losses_b': [], 'losses_f': [], 'time': []}
        self.min_loss = 1

        # Initialize the model
        self.model = Net(self.layers, self.X_ref.cpu().detach().numpy(), self.device).to(self.device)

    def Mseb(self):
        """Compute the boundary condition loss"""
        pred_lbc = self.model(self.X_lbc)
        pred_ubc = self.model(self.X_ubc)
        mseb = ((pred_lbc - 0) ** 2 + (pred_ubc - (1 - torch.exp(torch.tensor(-4.0)))) ** 2).mean()
        return mseb

    def TimeStepping(self):
        """Time stepping update"""
        X = self.X
        pred = self.model(X)
        u = pred
        self.U0 = u.detach()

    def Msef(self):
        """Compute the PDE residual loss"""
        X = self.X
        pred = self.model(X)
        u = pred
        u_x = fwd_gradients(u, X)
        u_xx = fwd_gradients(u_x, X)
        a = 0.008
        res = u_x * a + u_xx
        U1 = u
        R1 = res

        dtau = 100  # Time step size
        msef = 1 / dtau ** 2 * ((U1 - self.U0 + dtau * R1) ** 2).mean()
        return msef

    def Loss(self):
        """Compute the total loss"""
        mseb = self.Mseb()
        msef = self.Msef()
        loss = mseb + msef
        return loss, mseb, msef

    def ResidualPoint(self):
        """Generate PDE residual points"""
        self.X = torch.linspace(0, 10, 2000, device=self.device).reshape(-1, 1)
        self.X.requires_grad = True

    def train(self, epoch):
        """Train the model"""
        if len(self.log['time']) == 0:
            t1 = time.time()
        else:
            t1 = time.time() - self.log['time'][-1]

        # Adam optimizer with learning rate
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=3e-4)

        for i in range(epoch):
            self.ResidualPoint()
            self.TimeStepping()

            # Forward pass and loss calculation
            self.optimizer.zero_grad()
            self.loss, self.loss_b, self.loss_f = self.Loss()
            self.loss.backward()
            self.optimizer.step()

            # Record loss and time
            t2 = time.time()
            self.log['losses'].append(self.loss.item())
            self.log['losses_b'].append(self.loss_b.item())
            self.log['losses_f'].append(self.loss_f.item())
            self.log['time'].append(t2 - t1)

            # Print training progress
            print(f'Epoch {i + 1}/{epoch} - Total Loss: {self.loss.item():.6f}, Boundary Loss: {self.loss_b.item():.6f}, PDE Loss: {self.loss_f.item():.6f}')
            if (i % np.clip(int(epoch / 1000), 1, 1000) == 0) or (i == epoch - 1):
                print(f'{i}|{epoch} Total Loss={self.loss.item():.6f} PDE Loss={self.loss_f.item():.6f}')

# Exact solution function
def func(x):
    a = 0.008
    phi_d = -500
    x_cpu = x.detach().cpu().numpy()
    term1 = (1 - np.exp(a * phi_d)) * np.exp(a / 2 * (10 - x_cpu))
    sinh_ratio = np.sinh(a * x_cpu / 2) / np.sinh(a * 10 / 2)
    term2 = np.exp(a * phi_d)
    return torch.tensor(term1 * sinh_ratio, dtype=torch.float32)

# Main program
if __name__ == '__main__':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    layers = [1, 128, 128, 128, 1]
    tsonn = TSONN(layers, device)
    tsonn.train(10000)  # Set 10000 iterations

    # Save total loss to Excel file
    loss_df = pd.DataFrame({
        'Epoch': list(range(1, len(tsonn.log['losses']) + 1)),
        'Total Loss': tsonn.log['losses']
    })
    loss_df.to_excel('total_loss_silu.xlsx', index=False)
    print("Total loss values saved to total_loss.xlsx")

    # Print total loss statistics
    print("\n=== Total Loss Statistics ===")
    print(f"Total training epochs: {len(tsonn.log['losses'])}")
    print(f"Initial total loss: {tsonn.log['losses'][0]:.6f}")
    print(f"Final total loss: {tsonn.log['losses'][-1]:.6f}")
    print(f"Minimum total loss: {min(tsonn.log['losses']):.6f}")
    print(f"Average total loss: {np.mean(tsonn.log['losses']):.6f}")

    # Plot loss curve
    plt.figure(figsize=(12, 6))
    plt.plot(tsonn.log['losses'], label='Total Loss', color='blue')
    plt.plot(tsonn.log['losses_b'], label='Boundary Loss', color='orange')
    plt.plot(tsonn.log['losses_f'], label='PDE Loss', color='green')
    plt.yscale('log')
    plt.xlabel('Epochs')
    plt.ylabel('Loss Value (log scale)')
    plt.title('Training Loss Curve')
    plt.legend()
    plt.grid(True, which="both", ls="--")
    plt.annotate(f'Final Total Loss: {tsonn.log["losses"][-1]:.2e}',
                 xy=(len(tsonn.log['losses']), tsonn.log['losses'][-1]),
                 xytext=(len(tsonn.log['losses'])-2000, tsonn.log['losses'][-1]*2),
                 arrowprops=dict(facecolor='black', shrink=0.05))
    plt.savefig('loss_curve_annotated_tanh.png')
    plt.show()

    # Predict values at 101 points in the range [0, 10]
    x = torch.linspace(0, 10, 101).reshape(-1, 1).to(device)
    u_pred = tsonn.model(x).cpu().detach().numpy()
    u_exact = func(x.cpu()).detach().numpy()

    # Transform values - Add NaN check and handling
    u_pred_transformed = transform_values(torch.tensor(u_pred)).cpu().detach().numpy()
    u_exact_transformed = transform_values(torch.tensor(u_exact)).cpu().detach().numpy()

    # Check and handle NaN values
    mask = ~np.isnan(u_pred_transformed.flatten()) & ~np.isnan(u_exact_transformed.flatten())
    u_pred_transformed = u_pred_transformed.flatten()[mask]
    u_exact_transformed = u_exact_transformed.flatten()[mask]
    x_filtered = x.cpu().numpy().flatten()[mask]

    if len(u_pred_transformed) == 0:
        raise ValueError("All transformed values are NaN, please check the transform function and model output")

    # Compute evaluation metrics
    r2_transformed = r2_score(u_exact_transformed, u_pred_transformed)
    relative_l2_error_transformed = np.linalg.norm(u_exact_transformed - u_pred_transformed) / np.linalg.norm(u_exact_transformed)

    print(f"R² Score (transformed): {r2_transformed:.4f}")
    print(f"Relative L2 Error (transformed): {relative_l2_error_transformed:.4e}")

    # Save prediction results
    data = {
        'x': x_filtered,
        'True Value': u_exact.flatten()[mask],
        'Predicted Value': u_pred.flatten()[mask],
        'Transformed True Value': u_exact_transformed,
        'Transformed Predicted Value': u_pred_transformed
    }
    df = pd.DataFrame(data)
    df.to_excel('predictions_101_points_silu.xlsx', index=False)
    print("Prediction results saved to predictions_101_points.xlsx")

    # Plot comparison of solutions
    plt.figure(figsize=(12, 6))
    plt.plot(x_filtered, u_pred_transformed, label='Transformed Predicted u(x)')
    plt.plot(x_filtered, u_exact_transformed, label='Transformed Exact u(x)', linestyle='dashed')
    plt.title('Comparison of Transformed Predicted and Exact Solutions (Valid Points)')
    plt.xlabel('x')
    plt.ylabel('Transformed u')
    plt.legend()
    plt.grid(True)
    plt.savefig('solution_comparison.png')
    plt.show()
